In [218]:
import pandas as pd
import numpy as np
from datetime import datetime

In [219]:
df = pd.read_csv("../data/raw/Motor_Vehicle_Collisions_-_Crashes.csv")

# drop useless columns
df = df.drop(columns=["ZIP CODE", "LOCATION", "ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME", "COLLISION_ID"])

# convert "CRASH DATE" from object to datetime
df[["CRASH DATE"]] = pd.to_datetime(df[["CRASH DATE"]].stack()).unstack()

/tmp/ipykernel_273/2113288726.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw/Motor_Vehicle_Collisions_-_Crashes.csv")


In [220]:
# so that filtering out the date of crashes is a lot easier
df = df[df["CRASH DATE"] > datetime(2016, 12, 31)]
df = df[df["CRASH DATE"] < datetime(2020, 1, 1)]

# the time of the crash needs to be converted to a consistent format (e.g. 2:10 should be 02:10)
# the "CRASH TIME" column is located at index 1
for i in range(df.shape[0]):
    time = df.iat[i, 1]
    if len(time) == 4:
        time = f"0{time}"
        df.iat[i, 1] = time

df.head()


,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,...,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
20,2019-05-21,22:50,BROOKLYN,40.697540,-73.98312,0.0,0.0,0,0,0,...,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,�MBU,Taxi,NaN,NaN,NaN
1059,2019-04-17,00:49,NaN,40.651974,-73.86542,3.0,0.0,0,0,0,...,Following Too Closely,Unspecified,NaN,NaN,NaN,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
20389,2019-07-22,08:20,BROOKLYN,40.615433,-73.91388,0.0,0.0,0,0,0,...,Failure to Yield Right-of-Way,Failure to Yield Right-of-Way,NaN,NaN,NaN,Sedan,Sedan,NaN,NaN,NaN
33658,2019-10-19,17:20,NaN,NaN,NaN,0.0,0.0,0,0,0,...,Unsafe Lane Changing,Unspecified,NaN,NaN,NaN,Sedan,Sedan,NaN,NaN,NaN
38166,2017-12-09,20:05,NaN,NaN,NaN,1.0,0.0,1,0,0,...,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 674056 entries, 20 to 963176
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   CRASH DATE                     674056 non-null  datetime64[ns]
 1   CRASH TIME                     674056 non-null  object        
 2   BOROUGH                        429211 non-null  object        
 3   LATITUDE                       627643 non-null  float64       
 4   LONGITUDE                      627643 non-null  float64       
 5   NUMBER OF PERSONS INJURED      674040 non-null  float64       
 6   NUMBER OF PERSONS KILLED       674026 non-null  float64       
 7   NUMBER OF PEDESTRIANS INJURED  674056 non-null  int64         
 8   NUMBER OF PEDESTRIANS KILLED   674056 non-null  int64         
 9   NUMBER OF CYCLIST INJURED      674056 non-null  int64         
 10  NUMBER OF CYCLIST KILLED       674056 non-null  int64         
 11 

In [222]:
# I've taken a look at all the distinct values for vehicle type codes, and vehicles involving taxis are labelled "Taxi"
#df["VEHICLE TYPE CODE 1"].unique()

# find out how many crashes involve taxis
total_crashes = df.shape[0]
df = df.loc[(df["VEHICLE TYPE CODE 1"] == "Taxi") | (df["VEHICLE TYPE CODE 2"] == "Taxi")| (df["VEHICLE TYPE CODE 3"] == "Taxi") \
    | (df["VEHICLE TYPE CODE 4"] == "Taxi") | (df["VEHICLE TYPE CODE 5"] == "Taxi")]
taxi_crashes = df.shape[0]
taxi_crashes_perc = f"{round(taxi_crashes / total_crashes * 100, 2)}%"

# find out how likely a taxi trip could end up in a crash
print(taxi_crashes / 84598444 * 100)

df.sample(5)

0.0615696903361485


,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,...,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
749335,2017-11-17,17:00,NaN,40.826275,-73.85971,0.0,0.0,0,0,0,...,Turning Improperly,Unspecified,NaN,NaN,NaN,Taxi,Taxi,NaN,NaN,NaN
708323,2018-01-30,13:35,NaN,40.798576,-73.97316,0.0,0.0,0,0,0,...,Backing Unsafely,Unspecified,NaN,NaN,NaN,Sedan,Taxi,NaN,NaN,NaN
868438,2017-05-21,12:00,MANHATTAN,40.760822,-73.99832,0.0,0.0,0,0,0,...,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,Sedan,Taxi,NaN,NaN,NaN
772667,2017-10-16,22:00,NaN,40.770770,-73.91727,0.0,0.0,0,0,0,...,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,Sedan,Taxi,NaN,NaN,NaN
866319,2017-05-28,00:10,QUEENS,40.750000,-73.89924,0.0,0.0,0,0,0,...,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,Taxi,Taxi,NaN,NaN,NaN


In [223]:
# look at how the number of instances changes after filtering out non-taxi crashes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52087 entries, 20 to 963159
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   CRASH DATE                     52087 non-null  datetime64[ns]
 1   CRASH TIME                     52087 non-null  object        
 2   BOROUGH                        33764 non-null  object        
 3   LATITUDE                       48657 non-null  float64       
 4   LONGITUDE                      48657 non-null  float64       
 5   NUMBER OF PERSONS INJURED      52085 non-null  float64       
 6   NUMBER OF PERSONS KILLED       52083 non-null  float64       
 7   NUMBER OF PEDESTRIANS INJURED  52087 non-null  int64         
 8   NUMBER OF PEDESTRIANS KILLED   52087 non-null  int64         
 9   NUMBER OF CYCLIST INJURED      52087 non-null  int64         
 10  NUMBER OF CYCLIST KILLED       52087 non-null  int64         
 11  NUMBER OF MOT

In [224]:
tdf = pd.read_csv("../data/raw/twilight.csv")

# only the civil twilight data will be used. Why? Because although it occurs after sunset/before sunrise, 
# the sky is still bright enough that it doesn't require artificial lighting. When civil ends and nautical twilight begins
# (or the opposite during sunrise), the sky is then dark enough to require artificial lighting.
tdf = tdf.drop(columns=["begin_nau", "end_nau", "begin_astro", "end_astro"])

# convert time to 24hr format
for i in range(tdf.shape[0]):
    # this always ends in AM
    begin = f"0{tdf.iat[i, -2][0:4]}"
    # this always ends in PM
    end = f"{str(int(tdf.iat[i, -1][0]) + 12)}{tdf.iat[i, -2][1:4]}"

    tdf.iat[i, -2] = begin
    tdf.iat[i, -1] = end

tdf

,date,begin_civ,end_civ
0,"Tue, January 1",06:49,17:49
1,"Wed, January 2",06:49,17:49
2,"Thu, January 3",06:49,17:49
3,"Fri, January 4",06:49,17:49
4,"Sat, January 5",06:49,17:49
...,...,...,...
360,"Fri, December 27",06:48,17:48
361,"Sat, December 28",06:48,17:48
362,"Sun, December 29",06:48,17:48
363,"Mon, December 30",06:48,17:48


# DATA PREPROCESSING
## 1: Month
## 2: Daylight

In [225]:
# final data frame
fdf = df.iloc[:, [0, 1, 2, 3, 4]].copy().sort_values(by=["CRASH DATE", "CRASH TIME"])
fdf.head()

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE
957129,2017-01-01,00:01,NaN,40.865795,-73.920006
957053,2017-01-01,00:45,NaN,40.767815,-73.989510
953783,2017-01-01,01:25,MANHATTAN,40.749150,-73.995700
954351,2017-01-01,02:04,MANHATTAN,NaN,NaN
948003,2017-01-01,02:15,NaN,40.717987,-73.993810


In [226]:
# from the time data, extract info about whether it is night or day
# AND from the date data, extract info about its month
crash_time = fdf.iloc[:, 1].to_numpy()
daylight_start = tdf.iloc[:, -2].to_numpy()
daylight_end = tdf.iloc[:, -1].to_numpy()

month = []
daylight = []
for i in range(fdf.shape[0]):
    # DAYLIGHT
    # fdf.iloc[i, 0] is the crash date
    # yday = 0 for 1 Jan, or = 1 for 2 Jan, or = 364 for 31 Dec, etc, 
    # which will be used as the index for the twilight data
    yday = fdf.iloc[i, 0].timetuple().tm_yday - 1
        

    if (crash_time[i] < daylight_start[yday]) or (crash_time[i] > daylight_end[yday]):
        daylight.append("night")
    else:
        daylight.append("day")

    # MONTH
    month.append(fdf.iloc[i, 0].timetuple().tm_mon)

fdf["month"] = month
fdf["daylight"] = daylight

fdf.head(3)

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,month,daylight
957129,2017-01-01,00:01,NaN,40.865795,-73.920006,1,night
957053,2017-01-01,00:45,NaN,40.767815,-73.989510,1,night
953783,2017-01-01,01:25,MANHATTAN,40.749150,-73.995700,1,night


In [227]:
fdf["daylight"].value_counts()

day      31819
night    20268
Name: daylight, dtype: int64

It seems like crashes occur more during the day than it is during the night. However, we have to take into account that the number of vehicles outside is much higher during the day than night.

In [228]:
df["CRASH TIME"].value_counts()

00:00    761
17:00    610
16:00    598
18:00    542
14:00    528
        ... 
02:02      1
03:53      1
03:43      1
03:46      1
03:21      1
Name: CRASH TIME, Length: 1439, dtype: int64

This one is so funny, I guess the police or whoever's responsible filling in the crash time loves a nice even number. I mean, I can't blame them, I love even numbers too.

The outlier seems to be 00:00, which is probably because the crashes whose time are not recorded are automatically allocated to 00:00.

## 3: Sleep

In [229]:
# from the time data, extract info about whether it is during sleeping or waking hour
sleeping_hour = []
for i in range(fdf.shape[0]):
    if (crash_time[i] < "06:00") or (crash_time[i] > "23:00"):
        sleeping_hour.append("yes")
    else:
        sleeping_hour.append("no")

fdf["sleeping_hour"] = sleeping_hour
fdf.sample(3)

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,month,daylight,sleeping_hour
644800,2018-05-23,19:35,NaN,40.82815,-73.934910,5,day,no
531266,2018-10-20,20:30,MANHATTAN,40.74639,-73.977646,10,night,no
421247,2019-05-11,23:37,QUEENS,40.69808,-73.843445,5,night,yes


## 4: Location

In [230]:
import geopandas as gpd

# from tute2
sf = gpd.read_file("../data/raw/taxi_zones/taxi_zones.shp")
zones = pd.read_csv("../data/raw/taxi_zones/taxi+_zone_lookup.csv")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)

gdf.head(3)

,LocationID,Borough,Zone,service_zone,OBJECTID,Shape_Leng,Shape_Area,zone,borough,geometry
0,1,EWR,Newark Airport,EWR,1,0.116357,0.000782,Newark Airport,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,2,Queens,Jamaica Bay,Boro Zone,2,0.433470,0.004866,Jamaica Bay,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone,3,0.084341,0.000314,Allerton/Pelham Gardens,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."


In [233]:
# create POINT() geometry for the longitude and latitude of each crash
fdf = gpd.GeoDataFrame(
    fdf, geometry=gpd.points_from_xy(fdf.LONGITUDE, fdf.LATITUDE))
fdf.head()

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,month,daylight,sleeping_hour,geometry
957129,2017-01-01,00:01,NaN,40.865795,-73.920006,1,night,yes,POINT (-73.92001 40.86579)
957053,2017-01-01,00:45,NaN,40.767815,-73.989510,1,night,yes,POINT (-73.98951 40.76781)
953783,2017-01-01,01:25,MANHATTAN,40.749150,-73.995700,1,night,yes,POINT (-73.99570 40.74915)
954351,2017-01-01,02:04,MANHATTAN,NaN,NaN,1,night,yes,POINT EMPTY
948003,2017-01-01,02:15,NaN,40.717987,-73.993810,1,night,yes,POINT (-73.99381 40.71799)


In [234]:
# match every point to the zone and borough it belongs
gdf = gdf.iloc[:, [-3, -2, -1]]
fdf_new = gpd.sjoin(fdf, gdf, how="left", predicate='within')
fdf_new.head()

/tmp/ipykernel_273/3387784127.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...

  fdf_new = gpd.sjoin(fdf, gdf, how="left", predicate='within')


,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,month,daylight,sleeping_hour,geometry,index_right,zone,borough
957129,2017-01-01,00:01,NaN,40.865795,-73.920006,1,night,yes,POINT (-73.92001 40.86579),126.0,Inwood,Manhattan
957053,2017-01-01,00:45,NaN,40.767815,-73.989510,1,night,yes,POINT (-73.98951 40.76781),49.0,Clinton West,Manhattan
953783,2017-01-01,01:25,MANHATTAN,40.749150,-73.995700,1,night,yes,POINT (-73.99570 40.74915),67.0,East Chelsea,Manhattan
954351,2017-01-01,02:04,MANHATTAN,NaN,NaN,1,night,yes,POINT EMPTY,NaN,NaN,NaN
948003,2017-01-01,02:15,NaN,40.717987,-73.993810,1,night,yes,POINT (-73.99381 40.71799),147.0,Lower East Side,Manhattan


In [237]:
# all that's left to do is remove unnecessary variables after the join and a little bit more cleanup.
fdf = fdf_new.drop(columns=["BOROUGH", "LATITUDE", "LONGITUDE", "geometry"])
fdf = fdf.rename(columns={"CRASH DATE": "date", "CRASH TIME": "time", "index_right": "LocationID"})
fdf["LocationID"] = fdf["LocationID"].astype("Int64")
fdf.head()

,date,time,month,daylight,sleeping_hour,LocationID,zone,borough
957129,2017-01-01,00:01,1,night,yes,126,Inwood,Manhattan
957053,2017-01-01,00:45,1,night,yes,49,Clinton West,Manhattan
953783,2017-01-01,01:25,1,night,yes,67,East Chelsea,Manhattan
954351,2017-01-01,02:04,1,night,yes,<NA>,NaN,NaN
948003,2017-01-01,02:15,1,night,yes,147,Lower East Side,Manhattan


## 5: further attributes from the taxi data in preprocess.ipynb

## Wrapping up

In [310]:
# to analyse the number of crashes based on the attributes I've chosen, 
# I will use groupby to get the total number of crashes for each unique combination of attributes.
# And it seems like i will not use the zones as attributes, because they make the data look scarce and I don't think
# it's a useful application in real life anyway, because the zones are geographically small and most trips will
# go through multiple zones, so I believe it is more realistic to use the borough instead.
fdf_new = fdf.copy()
fdf_new = fdf_new.drop(columns=["time", "LocationID"]).rename(columns={"zone": "count"})
fdf_new = fdf_new.groupby(["date", "month", "daylight", "sleeping_hour", "borough"]).count().reset_index()
fdf_new.sort_values("count")

,date,month,daylight,sleeping_hour,borough,count
0,2017-01-01,1,day,no,Bronx,1
5206,2018-06-14,6,night,yes,Queens,1
5211,2018-06-15,6,night,no,Manhattan,1
5212,2018-06-15,6,night,no,Queens,1
5224,2018-06-16,6,night,yes,Queens,1
...,...,...,...,...,...,...
1554,2017-06-13,6,day,no,Manhattan,35
4994,2018-05-24,5,day,no,Manhattan,36
1704,2017-06-29,6,day,no,Manhattan,38
1159,2017-05-02,5,day,no,Manhattan,38


In [301]:
# save fdf to csv
fdf_new.to_csv("../data/curated/crime.csv", index=False)